# RNN을 이용한 텍스트 생성

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
text = '''
경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n'''

### 텍스트 전처리

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text]) # fit_transform 아님 주의 - 이렇게 하면 bag of words 생성됨

t.word_index    # 단어별 넘버링 확인

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [5]:
# 단어 집합의 크기 - keras.Tokenizer의 시작 인덱스가 1이기 때문에 0부터 시작해도 되도록 하나를 더해준다
vocab_size = len(t.word_index) + 1
vocab_size

12

In [6]:
t.texts_to_sequences(['그의 말이 법이다'])

[[6, 1, 7]]

In [7]:
# 학습에 사용될 샘플 시퀀스
sequences = []
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences

[[2],
 [2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6],
 [6, 1],
 [6, 1, 7],
 [8],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [8]:
max_len =max(len(s) for s in sequences)         # 데이터가 많아지면 가장 긴 것을 찾기 어려우므로, 수식 사용
max_len

6

### Padding
- 전체 샘플의 길이를 가장 긴 샘플의 길이로 패딩 : 여기서는 6,  
- 'pre'옵션을 주면 앞을 0으로 채움.

In [9]:
sequences = pad_sequences(sequences,maxlen=max_len, padding='pre')          # pre가 디폴트값
sequences

array([[ 0,  0,  0,  0,  0,  2],
       [ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

In [10]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [11]:
# y는 원핫인코딩 해야함. - to_categorical(y,vocab_size)로 해도 되지만 자동으로 하나 추가해서 맞춰준다.
Y = to_categorical(y)
Y[:3]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

### 모델 정의 
- Embedding을 먼저 - 열두개의 단어르 차원을 축소하고, 소수로 만들면 sparse하지 않으면서 데이터를 함축적으로 만들 수 있음  
- SimpleRNN

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense

임베딩이 제일 먼저 - 12개인 vocab_size를 4개로 낮출것이고, input_length는 최대 길이에서 y를 제외하므로 -1을 해줘야 한다.  
SimpleRNN - 유닛의 개수는 마음대로 

In [13]:
# Embedding = 4, SimpleRNN = 32
model = Sequential([
    Embedding(vocab_size,4, input_length=max_len-1),       
    SimpleRNN(32),
    Dense(vocab_size,activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 4)              48        
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                1184      
_________________________________________________________________
dense (Dense)                (None, 12)                396       
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


### 모델 설정

In [14]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:
history = model.fit(X,Y, epochs=200,verbose=0)
history.history['accuracy'][-1]     # 정확도

0.7857142686843872

### 모델 검증

In [16]:
from my_util import sentence_generation

In [21]:
# 모델, 토크나이저, 현재 단어, 반복할 횟수를 넣어준다.
sentence_generation(model, t, '경마장에',4)

'경마장에 말이 말이 뛰고 있다'

In [19]:
sentence_generation(model, t, '그의',2)

'그의 말이 법이다'

In [20]:
sentence_generation(model, t, '가는',5)

'가는 말이 고와야 오는 말이 곱다'

### 모델 변화  
- Embedding vector 갯수 : 2,4,6 
- RNN 노드 갯수 : 24,32,48

In [26]:
def execute_model(n_embed, n_node) : 
    model = Sequential([
    Embedding(vocab_size,n_embed, input_length=max_len-1),       
    SimpleRNN(n_node),
    Dense(vocab_size,activation='softmax')])

    model.compile(
        optimizer = 'adam',
        loss = 'categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(X,Y, epochs=200,verbose=0)
    print(f'n_embed = {n_embed}와 n_node = {n_node}의 정확도는 {history.history["accuracy"][-1]:4f}')
    print(sentence_generation(model, t, '경마장에',4))
    print(sentence_generation(model, t, '그의',2))
    print(sentence_generation(model, t, '가는',5))

In [27]:
for n_embed in [2,4,6]:
    for n_node in [24,32,48]:
        print("----------------------")
        execute_model(n_embed,n_node)

----------------------
n_embed = 2와 n_node = 24의 정확도는 0.785714
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
----------------------
n_embed = 2와 n_node = 32의 정확도는 0.714286
경마장에 말이 고와야 오는 말이
그의 말이 고와야
가는 말이 고와야 오는 말이 곱다
----------------------
n_embed = 2와 n_node = 48의 정확도는 0.785714
경마장에 말이 고와야 오는 말이
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
----------------------
n_embed = 4와 n_node = 24의 정확도는 0.714286
경마장에 말이 법이다 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
----------------------
n_embed = 4와 n_node = 32의 정확도는 0.857143
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
----------------------
n_embed = 4와 n_node = 48의 정확도는 0.785714
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
----------------------
n_embed = 6와 n_node = 24의 정확도는 0.785714
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
----------------------
n_embed = 6와 n_node = 32의 정확도는 0.857143
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
----------------------
n_embed = 6와 n_node = 48의 정확도는 0.857143
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다
